# Leitura dos dados de dengue

In [0]:
from pyspark.sql.functions import when, substring, col

df = spark.read.csv("/Volumes/workspace/aprendendocomizabela/dados_dengue/bruto/DENGBR25.csv", header=True)
print(df.columns)

# Colunas para análise
COLUNAS_A_SEREM_EXIBIDAS = ['DT_NOTIFIC', 'ID_AGRAVO', 'CLASSI_FIN', 'ID_MUNICIP', 'ANO_NASC', 'CS_SEXO', 'CS_GESTANT', 'evolucao', 'CS_ESCOL_N', 'DT_SIN_PRI', 'DT_OBITO', 'SG_UF', 'criterio','evolucao', 'SOROTIPO', 'NU_IDADE_N']
df = df.select(COLUNAS_A_SEREM_EXIBIDAS)

# De para dos valores originais

df = df.withColumn("C_CLASSI_FIN", when(df.CLASSI_FIN == 5, "Descartado")
                                .when(df.CLASSI_FIN == 10, "Dengue")
                                .when(df.CLASSI_FIN == 11, "Dengue com sinais de alarme")
                                .when(df.CLASSI_FIN == 12, "Dengue Grave")
                                .otherwise("Inconclusivo"))
                                
df = df.withColumn("CS_SEXO", when(df.CS_SEXO == "M", "Masculino")
                                .when(df.CS_SEXO == "F", "Feminino")
                                .otherwise(None))
                            
df = df.withColumn("C_CRITERIO", when(df.criterio == 1, "Laboratório")  
                        .when(df.criterio == 2, "Clínico-epidemiológico")
                        .when(df.criterio == 3, "Clínico")
                        .when(df.criterio == 9, "Ignorado")
                        .otherwise(None))
                            

df = df.withColumn("C_ESCOLARIDADE", when(df.CS_ESCOL_N == 1, "1_A_4_SERIE_INCOMPLETA")
                        .when(df.CS_ESCOL_N == 0, "ANALFABETO")  
                        .when(df.CS_ESCOL_N == 2, "4_SERIE_COMPLETA")
                        .when(df.CS_ESCOL_N == 5, "EM_INCOMPLETO")
                        .when(df.CS_ESCOL_N == 6, "EM_COMPLETO")
                        .when(df.CS_ESCOL_N == 7, "ES_INCOMPLETO")
                        .when(df.CS_ESCOL_N == 8, "ES_COMPLETO")
                        .when(df.CS_ESCOL_N == 10, "Ignorado")
                        .otherwise(None))

                              
df = df.withColumn("C_EVOLUCAO", when(df.evolucao == 1, "Cura")
                                .when(df.evolucao == 2, "Óbito por agravo")
                                .when(df.evolucao == 3, "Óbito por outras causas")
                                .when(df.evolucao == 4, "Óbito em investigação")
                                .when(df.evolucao == 9, "Ignorado")
                                .otherwise(None))

df = df.withColumn("C_SOROTIPO", when(df.SOROTIPO == 1, "1-DEN")
                                .when(df.SOROTIPO == 2, "2-DEN")
                                .when(df.SOROTIPO == 3, "3-DEN")
                                .when(df.SOROTIPO == 4, "4-DEN")
                                .otherwise(None))

df = df.withColumn("NU_IDADE_N", 
                   substring(col("NU_IDADE_N"), 3, 4))


                   

df = df.select(['ID_MUNICIP', 'NU_IDADE_N', 'SG_UF','ID_AGRAVO', 'DT_NOTIFIC', 'C_CLASSI_FIN', 'CS_SEXO', 'C_CRITERIO', 'C_ESCOLARIDADE', 'C_EVOLUCAO', 'C_SOROTIPO'])        

df.write.option("mergeSchema", "true").saveAsTable(f"workspace.aprendendocomizabela.tb_dengue", mode='overwrite')

df.toPandas()[:10000].to_csv("/Volumes/workspace/aprendendocomizabela/dados_dengue/tratado/df_casos_dengue.csv", index=False)

['TP_NOT', 'ID_AGRAVO', 'DT_NOTIFIC', 'SEM_NOT', 'NU_ANO', 'SG_UF_NOT', 'ID_MUNICIP', 'ID_REGIONA', 'ID_UNIDADE', 'DT_SIN_PRI', 'SEM_PRI', 'ANO_NASC', 'NU_IDADE_N', 'CS_SEXO', 'CS_GESTANT', 'CS_RACA', 'CS_ESCOL_N', 'SG_UF', 'ID_MN_RESI', 'ID_RG_RESI', 'ID_PAIS', 'DT_INVEST', 'ID_OCUPA_N', 'FEBRE', 'MIALGIA', 'CEFALEIA', 'EXANTEMA', 'VOMITO', 'NAUSEA', 'DOR_COSTAS', 'CONJUNTVIT', 'ARTRITE', 'ARTRALGIA', 'PETEQUIA_N', 'LEUCOPENIA', 'LACO', 'DOR_RETRO', 'DIABETES', 'HEMATOLOG', 'HEPATOPAT', 'RENAL', 'HIPERTENSA', 'ACIDO_PEPT', 'AUTO_IMUNE', 'DT_CHIK_S1', 'DT_CHIK_S2', 'DT_PRNT', 'RES_CHIKS1', 'RES_CHIKS2', 'RESUL_PRNT', 'DT_SORO', 'RESUL_SORO', 'DT_NS1', 'RESUL_NS1', 'DT_VIRAL', 'RESUL_VI_N', 'DT_PCR', 'RESUL_PCR_', 'SOROTIPO', 'HISTOPA_N', 'IMUNOH_N', 'HOSPITALIZ', 'DT_INTERNA', 'UF', 'MUNICIPIO', 'TPAUTOCTO', 'COUFINF', 'COPAISINF', 'COMUNINF', 'CLASSI_FIN', 'CRITERIO', 'DOENCA_TRA', 'CLINC_CHIK', 'EVOLUCAO', 'DT_OBITO', 'DT_ENCERRA', 'ALRM_HIPOT', 'ALRM_PLAQ', 'ALRM_VOM', 'ALRM_SANG', 

In [0]:
df.groupby('C_EVOLUCAO').count().display()


C_EVOLUCAO,count
Óbito por agravo,1406
null,341773
Ignorado,31934
Cura,1126284
Óbito por outras causas,329
Óbito em investigação,533


# Leitura da tabela de IBGE

In [0]:
from pyspark.sql.functions import when, col, substring
import pandas as pd 

df_pandas_municipio = pd.read_csv("/Volumes/workspace/aprendendocomizabela/dados_dengue/bruto/Cópia de RELATORIO_DTB_BRASIL_2024_DISTRITOS.csv", encoding='latin-1', on_bad_lines='skip', skiprows=6, sep=';')
df_municipios = spark.createDataFrame(df_pandas_municipio)

df_municipios = (
    df_municipios.withColumn(
                "ID_MUNICIPIO",
                substring(col('Código Município Completo').cast("string"), 0, 6)
            )      
)
df_municipios = (
            df_municipios.withColumnRenamed("UF", "ID_UF")
            .withColumnRenamed("Nome_UF", "NM_UF")
            .withColumnRenamed("Região Geográfica Intermediária", "ID_REGIAO_INTERMEDIARIA")      
            .withColumnRenamed("C_ID_MUNICIPIO", "ID_MUNICIPIO")
            .withColumnRenamed("Nome_Município", "NM_MUNICIPIO")
            .select(['ID_UF', 'NM_UF', 'ID_REGIAO_INTERMEDIARIA', 'ID_MUNICIPIO', 'NM_MUNICIPIO']))

file_path = "/Volumes/workspace/aprendendocomizabela/dados_dengue/tratado/"
df_municipios.write.saveAsTable(f"workspace.aprendendocomizabela.tb_municipios", mode='overwrite')
df_municipios.toPandas()[:10000].to_csv("/Volumes/workspace/aprendendocomizabela/dados_dengue/tratado/df_municipios.csv", index=False)
